In [ ]:
#!pip install tqdm

In [ ]:
from tqdm import tqdm
import melee
import os
import glob
import torch
import pickle
import json
import multiprocessing as mp
from util import *

In [2]:
# REPLACE WITH YOUR PATH NAME
source_path = "replays"
#source_path = "data"
# find all slp files in the data folder recursively
slp_paths = glob.glob(os.path.join(source_path, "**/*.slp"), recursive=True)
print(len(slp_paths))

# output_path = "data/processed"
output_path = "data/train_mini_50"

515


In [ ]:
def process_slp(slp):
    states = []
    actions = []
    game_id = os.path.basename(slp).split(".")[0]
    console = melee.Console(system="file", allow_old_version=False, path=slp)
    # console = melee.Console(system="file", path=slp)
    console.connect()
    while True:
        gs = console.step()
        if gs is None:
            break

        controller_state = gs.players[1].controller_state  # or whichever field holds the input
        act = get_controller_state(controller_state)

        obs = make_obs(gs, max_projectiles=5)
        states.append(obs)
        actions.append(act)

    # store the states and actions in a pkl file
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, f"{game_id}.pkl"), "wb") as f:
        pickle.dump({"states": states, "actions": actions}, f)

    return len(actions)

In [ ]:
# use multiprocessing to speed up the process
print(f'working with {mp.cpu_count()} cores')
with mp.Pool(processes=mp.cpu_count()) as pool:
    results = pool.map(process_slp, slp_paths[:50])
    # results = pool.map(process_slp, slp_paths)
    
metadata = {}
for k, v in zip(slp_paths, results):
    metadata[os.path.basename(k)] = v

with open(os.path.join(output_path, "metadata.json"), "w") as f:
    json.dump(metadata, f)


working with 12 cores
